In [1]:
import os
import ee
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd
import math
import skgstat as skg

In [2]:
# Create a project called "ee-sentinel-2temporal-nir"
# Google Account is jiawei.gao.polimi@gmail.com
ee.Authenticate(force = False)
ee.Initialize(project='ee-sentinel-2temporal-nir')

In [3]:
cwd = "c:\\Users\\m1865\\Desktop\\DISC"
cwd_Images_Raw = cwd + "\\Sentinel-2 Images Raw"
cwd_Images_Processed = cwd + "\\Sentinel-2 Images Processed"

In [4]:
date_Start = "2023-08-21"
date_End = "2023-09-01"
name_Site = "IT-SR2"
roi_Distance = 100

In [5]:
# Read shapefile
gdf_ROI = gpd.read_file(cwd_Images_Processed + "\\" + name_Site + "\\" + str(roi_Distance) + "m.shp")
EPSG_ROI = gdf_ROI.crs
# Convert it to 4326 which will be used by GEE
gdf_ROI_4326 = gdf_ROI.to_crs("EPSG:4326")
bounds = gdf_ROI_4326.bounds.values.reshape(-1)
roi = ee.Geometry.Rectangle([bounds[0],bounds[1],bounds[2],bounds[3]])
# Get its centroid and its coordinates in meters
centroid = gdf_ROI.centroid
centroid_x = centroid.get_coordinates().iloc[0,0]
centroid_y = centroid.get_coordinates().iloc[0,1]

In [37]:
bounds

array([10.29024478, 43.73147835, 10.29163159, 43.73248396])

In [6]:
# Get all the coordinates of pixel inside ROI
arr_x = np.arange(centroid_x - roi_Distance / 2, centroid_x + roi_Distance / 2 + 10, 10)
arr_y = np.arange(centroid_y - roi_Distance / 2, centroid_y + roi_Distance / 2 + 10, 10)
arr_xy = np.empty(shape = (len(arr_x)*len(arr_y),2), dtype = int)
z = 0
for j in range(len(arr_y)):
    for i in range(len(arr_x)):
        arr_xy[z,0] = arr_x[i]
        arr_xy[z,1] = arr_y[j]
        z = z + 1
# Create a new geodataframe
df_Points = pd.DataFrame({
    'x': arr_xy[:,0],
    'y': arr_xy[:,1]
})
gdf_Points = gpd.GeoDataFrame(df_Points, geometry = gpd.points_from_xy(df_Points['x'], df_Points['y'], crs=EPSG_ROI))
# gdf_Points_4326 = gdf_Points.to_crs("EPSG:4326")
# gdf_Points_4326['x'] = gdf_Points_4326.geometry.x
# gdf_Points_4326['y'] = gdf_Points_4326.geometry.y
# gdf_Points_4326
gdf_Points['x'] = gdf_Points.geometry.x
gdf_Points['y'] = gdf_Points.geometry.y
gdf_Points

,x,y,geometry
0,603915.0,4842865.0,POINT (603915 4842865)
1,603925.0,4842865.0,POINT (603925 4842865)
2,603935.0,4842865.0,POINT (603935 4842865)
3,603945.0,4842865.0,POINT (603945 4842865)
4,603955.0,4842865.0,POINT (603955 4842865)
...,...,...,...
116,603975.0,4842965.0,POINT (603975 4842965)
117,603985.0,4842965.0,POINT (603985 4842965)
118,603995.0,4842965.0,POINT (603995 4842965)
119,604005.0,4842965.0,POINT (604005 4842965)


# L1C B8 Rad

In [38]:
dataset_L1C_1 = ee.Image(
    ee.ImageCollection('COPERNICUS/S2_HARMONIZED')
    .filterBounds(roi)
    .filterDate(date_Start,date_End)
    # Pre-filter to get less cloudy granules.
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 50))
    .sort("system:time_start")
    .first()
)
L1C_info = dataset_L1C_1.getInfo()["properties"]
L1C_solar_z = L1C_info['MEAN_SOLAR_ZENITH_ANGLE']
L1C_offset_B8 = L1C_info['RADIO_ADD_OFFSET_B8']
L1C_solar_B8 = L1C_info['SOLAR_IRRADIANCE_B8']
L1C_u_B8 = L1C_info['REFLECTANCE_CONVERSION_CORRECTION']
L1C_B8_Ref = dataset_L1C_1.select(7).divide(10000)

In [39]:
list_L1C_B8_Rad = []
for i in range(gdf_Points.shape[0]):
    point = ee.Geometry.Point([gdf_Points.loc[i,'x'],gdf_Points.loc[i,'y']], proj='EPSG:32632')
    L1C_B8_Ref_repo = L1C_B8_Ref.reproject(crs = 'EPSG:32632')
    value = L1C_B8_Ref_repo.reduceRegion(
        reducer = ee.Reducer.first(),
        geometry = point,
        scale = 10,
    )
    pixelValues = value.getInfo()['B8']
    # radiance = reflectance * cos(radians(SunZenithAngle)) * solarIrradiance * U / pi
    pixelValues_toRad = pixelValues * math.cos(math.radians(L1C_solar_z)) * L1C_solar_B8 * L1C_u_B8 / math.pi
    list_L1C_B8_Rad.append(pixelValues_toRad)

# L2A NDVI

In [12]:
dataset_L2A_1 = ee.Image(
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterBounds(roi)
    .filterDate(date_Start,date_End)
    # Pre-filter to get less cloudy granules.
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 50))
    .sort("system:time_start")
    .first()
)
# L2A_info = dataset_L2A_1.getInfo()["properties"]
# L2A_offset_B4 = L2A_info['BOA_ADD_OFFSET_B4']
# L2A_offset_B8 = L2A_info['BOA_ADD_OFFSET_B8']
# L2A_B4_Ref = dataset_L2A_1.select(3).add(L2A_offset_B4).divide(10000)
# L2A_B8_Ref = dataset_L2A_1.select(7).add(L2A_offset_B8).divide(10000)
ndvi = dataset_L2A_1.normalizedDifference(['B8', 'B4']).rename('NDVI')

In [23]:
list_L2A_NDVI = []
for i in range(gdf_Points.shape[0]):
    point = ee.Geometry.Point([gdf_Points.loc[i,'x'],gdf_Points.loc[i,'y']], proj='EPSG:32632')
    ndvi_repo = ndvi.reproject(crs = 'EPSG:32632')
    value = ndvi_repo.reduceRegion(
        reducer = ee.Reducer.first(),
        geometry = point,
        scale = 10,
    )
    pixelValues = value.getInfo()['NDVI']
    list_L2A_NDVI.append(pixelValues)

# NIRv Rad

In [40]:
Nirv_Rad = np.multiply(list_L1C_B8_Rad,list_L2A_NDVI)
len(Nirv_Rad)

121

# Add NIRv Rad to geodataframe

In [41]:
gdf_Points['Rad'] = list_L1C_B8_Rad
gdf_Points['NDVI'] = list_L2A_NDVI
gdf_Points['NIRv Rad'] = Nirv_Rad
gdf_Points

,x,y,geometry,Rad,NDVI,NIRv Rad
0,603915.0,4842865.0,POINT (603915 4842865),55.540884,0.488786,27.147622
1,603925.0,4842865.0,POINT (603925 4842865),52.571213,0.650546,34.199993
2,603935.0,4842865.0,POINT (603935 4842865),56.825067,0.753695,42.828745
3,603945.0,4842865.0,POINT (603945 4842865),59.500446,0.746953,44.444061
4,603955.0,4842865.0,POINT (603955 4842865),57.627680,0.743780,42.862326
...,...,...,...,...,...,...
116,603975.0,4842965.0,POINT (603975 4842965),41.334618,0.675789,27.933499
117,603985.0,4842965.0,POINT (603985 4842965),41.281110,0.685775,28.309551
118,603995.0,4842965.0,POINT (603995 4842965),58.243018,0.787810,45.884454
119,604005.0,4842965.0,POINT (604005 4842965),53.935656,0.806559,43.502275


In [42]:
gdf_Points['NIRv Rad'].std() / gdf_Points['NIRv Rad'].mean()

0.10772625743844366

# Variogram

In [48]:
import skgstat as skg
band = gdf_Points['NIRv Rad'].to_numpy().reshape(-1)
# Get the maximum lag
diagonal_100 = 100 * (2 ** 0.5) / 2
bin_100 = np.arange(10,diagonal_100,10)
bin_100[-1]
V_100 = skg.Variogram(arr_xy, band, use_nugget=True, bin_func=bin_100, maxlag = bin_100[-1])
V_100.describe()

{'model': 'spherical',
 'estimator': 'matheron',
 'dist_func': 'euclidean',
 'normalized_effective_range': 3341.36508195497,
 'normalized_sill': 357.7930908407304,
 'normalized_nugget': 3.7921337502169876e-20,
 'effective_range': 47.733786885071,
 'sill': 18.80899940190609,
 'nugget': 1.9935052762527967e-21,
 'params': {'estimator': 'matheron',
  'model': 'spherical',
  'dist_func': 'euclidean',
  'bin_func': 'custom_bin_edges',
  'normalize': False,
  'fit_method': 'trf',
  'fit_sigma': None,
  'use_nugget': True,
  'maxlag': 70.0,
  'n_lags': 10,
  'verbose': False},
 'kwargs': {}}

# ONHOLD

In [104]:
dataset_L1C = (ee.ImageCollection('COPERNICUS/S2_HARMONIZED')
               .filterBounds(roi)
               .filterDate(date_Start,date_End)
               .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 50))
                .sort("system:time_start")  
)
imageList = dataset_L1C.toList(dataset_L1C.size())
image_2 = ee.Image(imageList.get(0))
image_2.getInfo()['id']

'COPERNICUS/S2_HARMONIZED/20230821T100601_20230821T101414_T32TPP'

In [35]:
dataset_L2A = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
               .filterBounds(roi)
               .filterDate(date_Start,date_End)
               .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 50))
                .sort("system:time_start")  
)
imageList = dataset_L2A.toList(dataset_L2A.size())
image_2 = ee.Image(imageList.get(2))
image_2.getInfo()

{'type': 'Image',
 'bands': [{'id': 'B1',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [1830, 1830],
   'crs': 'EPSG:32632',
   'crs_transform': [60, 0, 600000, 0, -60, 4900020]},
  {'id': 'B2',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32632',
   'crs_transform': [10, 0, 600000, 0, -10, 4900020]},
  {'id': 'B3',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32632',
   'crs_transform': [10, 0, 600000, 0, -10, 4900020]},
  {'id': 'B4',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32632',
   'crs_transform': [10, 0, 600000, 0, -10, 4900020]},
  {'id': 'B5',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min':